In [6]:
import os
import glob
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
df_site = pd.read_csv("/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/Filtered_AmeriFlux_Sites.csv",index_col=0)

In [4]:
data_list = glob.glob("/work/tadesse/beichen/ForDRI/Data/All_AmeriFlex_Data/"+"*.csv")
data_list.sort()

In [5]:
# for each site in the site dataframe
for idx, r in df_site.iterrows():
    site_id= r.loc["Site ID"]
    for d in data_list:
        if d.split("_")[3]==site_id:
            print(site_id," is under processing... ...")
            df = pd.read_csv(d, delimiter = ",",skiprows=2)
            df = df.replace([-9999], np.nan)
            if ['LE'] in df.columns.values:
                df_dropna = df.dropna(subset = ['H','LE'],axis = 0)
                df_dropna_cleaned = df_dropna.loc[:,['TIMESTAMP_START', 'TIMESTAMP_END','LE','H']]
                df_dropna_cleaned = df_dropna_cleaned.rename(columns={"TIMESTAMP_START": "TIMESTAMP_START", "TIMESTAMP_END": "TIMESTAMP_END", "LE": "LE",
                                            "H": "H"})
            elif ['LE_1_1_1'] in df.columns.values:
                df_dropna = df.dropna(subset = ['H_1_1_1','LE_1_1_1'],axis = 0)
                df_dropna_cleaned = df_dropna.loc[:,['TIMESTAMP_START', 'TIMESTAMP_END','LE_1_1_1','H_1_1_1']]
                df_dropna_cleaned = df_dropna_cleaned.rename(columns={"TIMESTAMP_START": "TIMESTAMP_START", "TIMESTAMP_END": "TIMESTAMP_END", "LE_1_1_1": "LE",
                                            "H_1_1_1": "H"})
            else:
                print(df.columns)
            
            # filter the data if its flux measurement is lower than 50
            df_dropna_cleaned = df_dropna_cleaned.loc[(df_dropna_cleaned['LE']>50)&(df_dropna_cleaned['H']>50),:]
            df_dropna_cleaned.to_csv("/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/Cleaned_Flux_Data/"+site_id+".csv")
            
            del df,df_dropna,df_dropna_cleaned
    

US-xTA  is under processing... ...
US-Uaf  is under processing... ...
US-SRM  is under processing... ...
US-Cst  is under processing... ...
US-Ton  is under processing... ...
US-NR1  is under processing... ...
US-SP1  is under processing... ...
US-xJE  is under processing... ...
US-xPU  is under processing... ...
US-Rws  is under processing... ...
US-MMS  is under processing... ...
US-xUK  is under processing... ...
US-Ho2  is under processing... ...
US-xSE  is under processing... ...
US-Ha2  is under processing... ...
US-UMB  is under processing... ...
US-MOz  is under processing... ...
US-Bar  is under processing... ...
US-Slt  is under processing... ...
US-Vcm  is under processing... ...
US-NC2  is under processing... ...
US-Oho  is under processing... ...
US-Me2  is under processing... ...
US-SSH  is under processing... ...
PR-xGU  is under processing... ...
US-Akn  is under processing... ...
US-xRN  is under processing... ...
US-EA5  is under processing... ...
US-xNQ  is under pro

In [ ]:
flux_data_list = glob.glob("/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/Cleaned_Flux_Data/"+"*.csv")
flux_data_list.sort()
for flux in flux_data_list:
    fname = os.path.basename(flux)
    print(fname[:-4]," is under processing... ...")
    df = pd.read_csv(flux,index_col=0)
    df['TIMESTAMP_START'] = df['TIMESTAMP_START'].map(lambda x: int(datetime.strptime(str(x)[:8], "%Y%m%d").strftime("%Y%j")[:4])*100+int(datetime.strptime(str(x)[:8], "%Y%m%d").strftime("%Y%j")[-3:])//7+1)
    start_year = df.TIMESTAMP_START.values[0]//100
    end_year = df.TIMESTAMP_START.values[-1]//100
    print(start_year,end_year)
    weekly_index = []
    for i in range(start_year,end_year+1):
        for j in range(1,53):
            if j<10:
                j='0'+str(j)
            else:
                j=str(j)
            out = int(str(i)+str(j))
            weekly_index.append(out)
    df_out = pd.DataFrame(np.nan,index = weekly_index, columns = ["LE","H"])
    df = df.drop(['TIMESTAMP_END'],axis = 1)
    df = df.groupby('TIMESTAMP_START').sum()
    for idx in df.index.values:
        df_out.loc[idx,:]=df.loc[idx,:].values
    df_out = df_out.iloc[:(end_year-start_year+1)*52,:]
    df_out["Bowen_ratio"] = df_out.loc[:,"H"].values/df_out.loc[:,"LE"].values
    df_out["Log_Bowen_ratio"] = np.log10(df_out.loc[:,"Bowen_ratio"].values)
    reshaped_logbr = df_out.Log_Bowen_ratio.values.reshape(end_year-start_year+1,52)
    mean=np.nanmean(reshaped_logbr, axis = 0)
    std = np.nanstd(reshaped_logbr, axis = 0)
    std_logbr = (mean - reshaped_logbr)/std
    df_out["STD_Log_Bowen_ratio"] = std_logbr.flatten()
    df_out.to_csv("/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/Bowen_Ratio/"+fname)
    
    del df_out, df